most recent as of 10/23/2023

# init packages

In [2]:
## Link: https://github.com/WJMatthew/WESAD/blob/master/data_wrangling.py
import os
import re
import graphviz
import io
import pickle
import numpy as np
import dask as dd
import dask.dataframe as dd
import pandas as pd
import scipy.stats
#import seaborn as sns
import scipy.signal as scisig
#from xgboost import XGBClassifier
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import matplotlib as mpl
import heartpy as hp
import biosppy
import neurokit2 as nk
from heartpy.datautils import *
from heartpy.peakdetection import *
mpl.rcParams['agg.path.chunksize'] = 10000
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn.feature_selection import mutual_info_classif
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report

In [3]:
256/4

64.0

## Data Imports

# Feature Extraction

## Parameters

In [4]:
# E4 (wrist) Sampling Frequencies
fs_dict = {'ACC': 64, 'BVP': 64, 'EDA': 64, 'TEMP': 64, 'label': 64, 'Resp': 64, 'ECG': 64, 
           'chest': 64}
# Window size
WINDOW_IN_SECONDS = 60
#rot_anx_dict = {'calibration': 0, 'LA': 1, 'HA': 2}
#calib_label = {'not calibration':0, 'calibration':1, 'cpt':2,'meditation':3}
# Labels
#label_dict = {'baseline': 1, 'stress': 2, 'amusement': 0} # WESAD labels
label_dict = {'calibration': 0, 'low anxiety': 1, 'high anxiety': 2} # RADWear labels 
# Int to label mappings
#int_to_label = {1: 'low anxiety', 2: 'high anxiety', 0: 'calibration'} WESAD labels
# Feature names
feat_names = None
# Where to save the data
savePath = '/projects/bbnp/Vansh/data/RADWear/output'

# Where to get the data
subject_feature_path = '/subject_feats'

if not os.path.exists(savePath):
    os.makedirs(savePath)
if not os.path.exists(savePath + subject_feature_path):
    os.makedirs(savePath + subject_feature_path)

## Class to Store Subject Data

In [5]:
# Class to store the data for each subject
class SubjectData:

    def __init__(self, main_path, subject_number):
        self.name = f'S{subject_number}'
        self.subject_keys = ['signal', 'label', 'subject']
        self.signal_keys = ['chest', 'wrist']
        self.chest_keys = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
        self.wrist_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        with open(os.path.join(main_path, self.name) + '/' + self.name + '.pkl', 'rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.labels = self.data['label']

    def get_wrist_data(self):
        data = self.data['signal']['wrist']
        data.update({'ACC_C': self.data['signal']['chest']['ACC'],
                     'ECG_C': self.data['signal']['chest']['ECG'],
                     'EDA_C': self.data['signal']['chest']['EDA'],
                     'EMG_C': self.data['signal']['chest']['EMG'],
                     'Resp_C': self.data['signal']['chest']['Resp'],
                     'Temp_C': self.data['signal']['chest']['Temp']
                     })
        return data

    def get_chest_data(self):
        return self.data['signal']['chest']
    
    def extract_features(self):  # only wrist
        results = \
            {
                key: get_statistics(self.get_wrist_data()[key].flatten(), self.labels, key)
                for key in self.wrist_keys
            }
        return results

## ACC

## Helper Functions

### Lowpass

In [6]:
# https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/load_files.py
def butter_lowpass(cutoff, fs, order=5):
    # Filtering Helper functions
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scisig.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

### Lowpass Filter

In [7]:
def butter_lowpass_filter(data, cutoff, fs, order=5):
    # Filtering Helper functions
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = scisig.lfilter(b, a, data)
    return y

### Slope Calculator

In [8]:
def get_slope(series):
    linreg = scipy.stats.linregress(np.arange(len(series)), series )
    slope = linreg[0]
    return slope

### Mean/Std/Min/Max

In [9]:
def get_window_stats(data, label=-1):
    mean_features = np.mean(data)
    std_features = np.std(data)
    min_features = np.amin(data)
    max_features = np.amax(data)

    features = {'mean': mean_features, 'std': std_features, 'min': min_features, 'max': max_features,
                'label': label}
    return features

### Absolute Integral

In [10]:
def get_absolute_integral(x):
    return np.sum(np.abs(x))

### Dynamic Range

In [11]:
def get_dynamic_range(x):
    return np.max(x) / np.min(x)

### Peak Frequency (Periodogram)

In [12]:
def get_peak_freq(x):
    f, Pxx = scisig.periodogram(x, fs=64)
    psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
    peak_freq = psd_dict[max(psd_dict.keys())]
    return peak_freq

### Respiration Features

In [13]:
def get_resp_features(resp_data):
    resp_rate, filtered, zeros, resp_rate_ts, resp_rate = biosppy.signals.resp.resp(resp_data, sampling_rate=64, show=False)
    extremas, values = biosppy.signals.tools.find_extrema(signal=filtered, mode='both')
    inhal_durations = []
    exhal_durations = []
    last_index = 0
    for i in range(len(extremas)):
        if values[i] * values[last_index] < 0:
            if values[last_index] < 0:
                inhal_durations.append((extremas[i] - extremas[last_index]) / 64)
            else:
                exhal_durations.append((extremas[i] - extremas[last_index]) / 64)
            last_index = i
    return np.mean(resp_rate), np.mean(inhal_durations), np.std(inhal_durations), np.mean(exhal_durations), np.std(exhal_durations), np.sum(inhal_durations) / np.sum(exhal_durations)

### FIR Filter

In [14]:
# https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/AccelerometerFeatureExtractionScript.py
def filterSignalFIR(eda, cutoff=0.4, numtaps=64):
    f = cutoff / (fs_dict['ACC'] / 2.0)
    FIR_coeff = scisig.firwin(numtaps, f)

    return scisig.lfilter(FIR_coeff, 1, eda)

## ECG

In [15]:
# Calculating window statistics for ECGs
def get_window_stats_ecg(data, label=-1):
    wd, m = hp.process(data['ECG'].dropna().reset_index(drop=True), 64)
    return {'bpm': m['bpm'], 'ibi': m['ibi'], 'sdnn': m['sdnn'], 'sdsd': m['sdsd'], 
            'rmssd': m['rmssd'], 'pnn20': m['pnn20'], 'pnn50': m['pnn50']}

In [16]:
def compute_features_chest(ch_data_dict, labels, norm_type=None):
    ecg_df = pd_old.DataFrame(ch_data_dict['ECG'], columns=['ECG'])
    
    # Adding index for combination due to different sampling frequencies
    ecg_df.index = [(1 / fs_dict['ECG']) * i for i in range(len(ecg_df))]
    
    # Change index to datetime
    ecg_df.index = pd_old.to_datetime(ecg_df.index, unit='s')
    
    return ecg_df

## Compute Features

In [17]:
# Computes features for wrist
def compute_features(e4_data_dict, ch_data_dict, labels, norm_type=None):
    # Dataframes for each sensor type
    eda_df = pd_old.DataFrame(e4_data_dict['EDA'], columns=['EDA'])
    bvp_df = pd_old.DataFrame(e4_data_dict['BVP'], columns=['BVP'])
    acc_df = pd_old.DataFrame(e4_data_dict['ACC'], columns=['ACC_x', 'ACC_y', 'ACC_z'])
    temp_df = pd_old.DataFrame(e4_data_dict['TEMP'], columns=['TEMP'])
    label_df = pd_old.DataFrame(labels, columns=['label'])
    resp_df = pd_old.DataFrame(e4_data_dict['Resp_C'], columns=['Resp_C'])
    acc_c_df = pd_old.DataFrame(e4_data_dict['ACC_C'], columns=['ACC_x_C', 'ACC_y_C', 'ACC_z_C'])
    ecg_c_df = pd_old.DataFrame(e4_data_dict['ECG_C'], columns=['ECG_C'])
    eda_c_df = pd_old.DataFrame(e4_data_dict['EDA_C'], columns=['EDA_C'])
    emg_c_df = pd_old.DataFrame(e4_data_dict['EMG_C'], columns=['EMG_C'])
    resp_c_df = pd_old.DataFrame(e4_data_dict['Resp_C'], columns=['Resp_C'])
    temp_c_df = pd_old.DataFrame(e4_data_dict['Temp_C'], columns=['Temp_C'])

    # Filter EDA
    eda_df['EDA'] = butter_lowpass_filter(eda_df['EDA'], 1.0, fs_dict['EDA'], 6)
    eda_c_df['EDA_C'] = butter_lowpass_filter(eda_c_df['EDA_C'], 1.0, fs_dict['chest'], 6)

    
    eda_data = nk.eda_phasic(nk.standardize(eda_df['EDA']), sampling_rate=fs_dict['EDA'])
    eda_df['EDA_SCR'] = eda_data['EDA_Phasic']
    eda_df['EDA_SCL'] = eda_data['EDA_Tonic']
    eda_data_c = nk.eda_phasic(nk.standardize(eda_c_df['EDA_C']), sampling_rate=fs_dict['chest'])
    eda_c_df['EDA_SCR_C'] = eda_data_c['EDA_Phasic']
    eda_c_df['EDA_SCL_C'] = eda_data_c['EDA_Tonic']
    
    # Filter ACM
    for _ in acc_df.columns:
        acc_df[_] = filterSignalFIR(acc_df.values)
    for _ in acc_c_df.columns:
        acc_c_df[_] = filterSignalFIR(acc_c_df.values)

    # Adding indices for combination due to differing sampling frequencies
    eda_df.index = [(1 / fs_dict['EDA']) * i for i in range(len(eda_df))]
    bvp_df.index = [(1 / fs_dict['BVP']) * i for i in range(len(bvp_df))]
    acc_df.index = [(1 / fs_dict['ACC']) * i for i in range(len(acc_df))]
    temp_df.index = [(1 / fs_dict['TEMP']) * i for i in range(len(temp_df))]
    label_df.index = [(1 / fs_dict['label']) * i for i in range(len(label_df))]
    resp_df.index = [(1 / fs_dict['Resp']) * i for i in range(len(resp_df))]
    acc_c_df.index = [(1 / fs_dict['chest']) * i for i in range(len(acc_c_df))]
    ecg_c_df.index = [(1 / fs_dict['chest']) * i for i in range(len(ecg_c_df))]
    eda_c_df.index = [(1 / fs_dict['chest']) * i for i in range(len(eda_c_df))]
    emg_c_df.index = [(1 / fs_dict['chest']) * i for i in range(len(emg_c_df))]
    resp_c_df.index = [(1 / fs_dict['chest']) * i for i in range(len(resp_c_df))]
    temp_c_df.index = [(1 / fs_dict['chest']) * i for i in range(len(temp_c_df))]

    # Change indices to datetime
    eda_df.index = pd_old.to_datetime(eda_df.index, unit='s')
    bvp_df.index = pd_old.to_datetime(bvp_df.index, unit='s')
    temp_df.index = pd_old.to_datetime(temp_df.index, unit='s')
    acc_df.index = pd_old.to_datetime(acc_df.index, unit='s')
    label_df.index = pd_old.to_datetime(label_df.index, unit='s')
    resp_df.index = pd_old.to_datetime(resp_df.index, unit='s')
    acc_c_df.index = pd_old.to_datetime(acc_c_df.index, unit='s')
    ecg_c_df.index = pd_old.to_datetime(ecg_c_df.index, unit='s')
    eda_c_df.index = pd_old.to_datetime(eda_c_df.index, unit='s')
    emg_c_df.index = pd_old.to_datetime(emg_c_df.index, unit='s')
    resp_c_df.index = pd_old.to_datetime(resp_c_df.index, unit='s')
    temp_c_df.index = pd_old.to_datetime(temp_c_df.index, unit='s')

    # New EDA features
#     r, p, t, l, d, e, obj = eda_stats(eda_df['EDA'])
#     eda_df['EDA_phasic'] = r
#     eda_df['EDA_smna'] = p
#     eda_df['EDA_tonic'] = t

    # Getting ECG features
    ecg_df = compute_features_chest(ch_data_dict, labels, norm_type=None)
        
    # Combined dataframe - not used yet
    df = eda_df.join(bvp_df, how='outer')
    df = df.join(temp_df, how='outer')
    df = df.join(acc_df, how='outer')
    df = df.join(label_df, how='outer')
    df = df.join(ecg_df, how='outer')
    df = df.join(eda_c_df, how='outer')
    df = df.join(acc_c_df, how='outer')
    df = df.join(emg_c_df, how='outer')
    df = df.join(resp_c_df, how='outer')
    df = df.join(temp_c_df, how='outer')
    df['label'] = df['label'].fillna(method='bfill')
    df.reset_index(drop=True, inplace=True)
    print(df.isna().sum())
    if norm_type == 'std':
        # std norm
        df = (df - df.mean()) / df.std()
    elif norm_type == 'minmax':
        # minmax norm
        df = (df - df.min()) / (df.max() - df.min())

    # Groupby
    grouped = df.groupby('label')
    baseline = grouped.get_group(1)
    stress = grouped.get_group(2)
    amusement = grouped.get_group(3)
    return grouped, baseline, stress, amusement

## Get Samples

In [18]:
def get_samples(data, n_windows, label):
    global feat_names
    global WINDOW_IN_SECONDS

    samples = []
    # Using label freq (64 Hz) as our reference frequency due to it being the largest
    # and thus encompassing the lesser ones in its resolution.
    window_len = fs_dict['label'] * WINDOW_IN_SECONDS

    for i in range(n_windows):
        # Get window of data
        w = data[window_len * i: window_len * (i + 1)]

        # Add/Calc rms acc
        w = pd_old.concat([get_net_accel(w), w])
        cols = list(w.columns)
        cols[0] = 'net_acc'
        w.columns = cols
        
        w = pd_old.concat([get_net_accel_C(w), w])
        cols = list(w.columns)
        cols[0] = 'net_acc_C'
        w.columns = cols
        
        # Calculate stats for window
        wstats = get_window_stats(data=w, label=label)
        
        # Calculate stats for window (ECG)
        wstats_ecg = get_window_stats_ecg(data=w, label=label)
        
        # Seperating sample and label
        x = pd_old.DataFrame(wstats).drop('label', axis=0)
        y = x['label'][0]
        x.drop('label', axis=1, inplace=True)
        
        feat_names = None
        if feat_names is None:
            feat_names = []
            for row in x.index:
                for col in x.columns:
                    feat_names.append('_'.join([str(row), str(col)]))

        # sample df
        wdf = pd_old.DataFrame(x.values.flatten()).T
        wdf.columns = feat_names
        wdf = pd_old.concat([wdf, pd_old.DataFrame({'label': y}, index=[0])], axis=1)
        
        # More feats
        wdf['BVP_peak_freq'] = get_peak_freq(w['BVP'].dropna())
        
        # Add more features here
        # ACC (w and c)
        wdf['net_acc_abs_integral'] = get_absolute_integral(w['net_acc'].dropna())
        wdf['ACC_x_abs_integral'] = get_absolute_integral(w['ACC_x'].dropna())
        wdf['ACC_y_abs_integral'] = get_absolute_integral(w['ACC_y'].dropna())
        wdf['ACC_z_abs_integral'] = get_absolute_integral(w['ACC_z'].dropna())
        wdf['net_acc_C_abs_integral'] = get_absolute_integral(w['net_acc_C'].dropna())
        wdf['ACC_x_C_abs_integral'] = get_absolute_integral(w['ACC_x_C'].dropna())
        wdf['ACC_y_C_abs_integral'] = get_absolute_integral(w['ACC_y_C'].dropna())
        wdf['ACC_z_C_abs_integral'] = get_absolute_integral(w['ACC_z_C'].dropna())
        wdf['ACC_x_peak_freq'] = get_peak_freq(w['ACC_x'].dropna())
        wdf['ACC_y_peak_freq'] = get_peak_freq(w['ACC_y'].dropna())
        wdf['ACC_z_peak_freq'] = get_peak_freq(w['ACC_z'].dropna())
        wdf['ACC_x_C_peak_freq'] = None if len(w['ACC_x_C'].dropna()) == 0 else get_peak_freq(w['ACC_x_C'].dropna())
        wdf['ACC_y_C_peak_freq'] = None if len(w['ACC_y_C'].dropna()) == 0 else get_peak_freq(w['ACC_y_C'].dropna())
        wdf['ACC_z_C_peak_freq'] = None if len(w['ACC_z_C'].dropna()) == 0 else get_peak_freq(w['ACC_z_C'].dropna())
        
        # ECG
        for key in wstats_ecg.keys():
            wdf['ECG_'+key] = wstats_ecg[key]
        
        # EDA(w and c)
        wdf['EDA_slope'] = get_slope(w['EDA'].dropna())
        wdf['EDA_C_slope'] = None if len(w['ACC_z_C'].dropna()) == 0 else get_slope(w['EDA_C'].dropna())
        wdf['EDA_drange'] = get_dynamic_range(w['EDA'].dropna())
        wdf['EDA_C_drange'] = None if len(w['EDA_C'].dropna()) == 0 else get_dynamic_range(w['EDA_C'].dropna())

        # EMG(c)
        wdf['EMG_drange'] = get_dynamic_range(w['EMG_C'].dropna())
        wdf['EMG_abs_integral'] = get_absolute_integral(w['EMG_C'].dropna())
        # RESP(c)
        
        if len(w['Resp_C'].dropna()) > 0:
            wdf['Resp_C_rate'], wdf['Resp_C_Inhal_mean'], wdf['Resp_C_Inhal_std'], wdf['Resp_C_Exhal_mean'], wdf['Resp_C_Exhal_std'], wdf['Resp_C_I/E'] = get_resp_features(w['Resp_C'].dropna())

        # TEMP(w and c)
        wdf['TEMP_drange'] = get_dynamic_range(w['TEMP'].dropna())
        wdf['TEMP_C_drange'] = None if len(w['Temp_C'].dropna()) == 0 else get_dynamic_range(w['Temp_C'].dropna())
        wdf['TEMP_slope'] = get_slope(w['TEMP'].dropna())
        wdf['TEMP_C_slope'] = None if len(w['Temp_C'].dropna()) == 0 else get_slope(w['Temp_C'].dropna())
        
        samples.append(wdf)

    return pd_old.concat(samples)

## Make Patient Data

In [19]:
def make_patient_data(subject_id):
    global savePath
    global WINDOW_IN_SECONDS

    # Make subject data object for Sx
    subject = SubjectData(main_path='data/WESAD', subject_number=subject_id)

    # Empatica E4 data - now with resp
    e4_data_dict = subject.get_wrist_data()

    # Chest data
    ch_data_dict = subject.get_chest_data()
    
    # norm type
    norm_type = None

    # The 3 classes we are classifying
    grouped, baseline, stress, amusement = compute_features(e4_data_dict, ch_data_dict, subject.labels, norm_type)

    # print(f'Available windows for {subject.name}:')
    n_baseline_wdws = int(len(baseline) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    n_stress_wdws = int(len(stress) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    n_amusement_wdws = int(len(amusement) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    # print(f'Baseline: {n_baseline_wdws}\nStress: {n_stress_wdws}\nAmusement: {n_amusement_wdws}\n')

    #
    baseline_samples = get_samples(baseline, n_baseline_wdws, 1)
#     for col in baseline_samples.columns:
#         print(col)
     # Downsampling
    # baseline_samples = baseline_samples[::2]
    stress_samples = get_samples(stress, n_stress_wdws, 2)
    amusement_samples = get_samples(amusement, n_amusement_wdws, 0)

    all_samples = pd_old.concat([baseline_samples, stress_samples, amusement_samples])
    all_samples = pd_old.concat([all_samples.drop('label', axis=1), pd_old.get_dummies(all_samples['label'])], axis=1)
    # Save file as csv
    all_samples.to_csv(f'{savePath}{subject_feature_path}/S{subject_id}_feats.csv')

    subject = None

## Combine Patient Files

In [20]:
import pandas as pd
def combine_files(subjects):
    df_list = []
    for s in subjects:
        df = pd.read_csv(f'{savePath}{subject_feature_path}/S{s}_feats.csv', index_col=0)
        df['subject'] = s
        df_list.append(df)

    df = pd_old.concat(df_list)

    df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str)).apply(lambda x: x.index('1'))
    df.drop(['0', '1', '2'], axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)

    df.to_csv(f'{savePath}/may14_feats4.csv')

    counts = df['label'].value_counts()
    print('Number of samples per class:')
    for label, number in zip(counts.index, counts.values):
        print(f'{int_to_label[label]}: {number}')


if __name__ == '__main__':
    
    #subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17] # WESAD
     # part of RADWear. others like 5, 4, 5, 16 18 19 20 etc
    '''
    for patient in subject_ids:
        print(f'Processing data for S{patient}...')
        make_patient_data(patient)
    '''
    #combine_files(subject_ids)
    print('Processing complete.')

Processing complete.


## Add Demographic Data

In [21]:
class rparser:
    # Code adapted from https://github.com/arsen-movsesyan/springboard_WESAD/blob/master/parsers/readme_parser.py
    VALUE_EXTRACT_KEYS = {
        "age": {
            'search_key': 'Age',
            'delimiter': ':'
        },
        "height": {
            'search_key': 'Height',
            'delimiter': ':'
        },
        "weight": {
            'search_key': 'Weight',
            'delimiter': ':'
        },
        "gender": {
            'search_key': 'Gender',
            'delimiter': ':'
        },
        "dominant_hand": {
            'search_key': 'Dominant',
            'delimiter': ':'
        },
        "coffee_today": {
            'search_key': 'Did you drink coffee today',
            'delimiter': '? '
        },
        "coffee_last_hour": {
            'search_key': 'Did you drink coffee within the last hour',
            'delimiter': '? '
        },
        "sport_today": {
            'search_key': 'Did you do any sports today',
            'delimiter': '? '
        },
        "smoker": {
            'search_key': 'Are you a smoker',
            'delimiter': '? '
        },
        "smoke_last_hour": {
            'search_key': 'Did you smoke within the last hour',
            'delimiter': '? '
        },
        "feel_ill_today": {
            'search_key': 'Do you feel ill today',
            'delimiter': '? '
        }
    }
    
    #DATA_PATH = 'data/WESAD/'
    DATA_PATH = '/mnt/c/Users/alkurdi/Desktop/Vansh/data/RADWear'

    parse_file_suffix = '_readme.txt'
        
    
    def __init__(self):
        
        self.readme_locations = {subject_directory: self.DATA_PATH + subject_directory + '/' 
                          for subject_directory in os.listdir(self.DATA_PATH)
                              if re.match('^S[0-9]{1,2}$', subject_directory)}
        
        # Check if parsed readme file is available ( should be as it is saved above )
        if not os.path.isfile('data/readmes.csv'):
            print('Parsing Readme files')
            self.parse_all_readmes()
        else:
            print('Files already parsed.')
            
        self.merge_with_feature_data()
        
        
    def parse_readme(self, subject_id):
        with open(self.readme_locations[subject_id] + subject_id + self.parse_file_suffix, 'r') as f:

            x = f.read().split('\n')

        readme_dict = {}

        for item in x:
            for key in self.VALUE_EXTRACT_KEYS.keys():
                search_key = self.VALUE_EXTRACT_KEYS[key]['search_key']
                delimiter = self.VALUE_EXTRACT_KEYS[key]['delimiter']
                if item.startswith(search_key):
                    d, v = item.split(delimiter)
                    readme_dict.update({key: v})
                    break
        return readme_dict


    def parse_all_readmes(self):
        
        dframes = []

        for subject_id, path in self.readme_locations.items():
            readme_dict = self.parse_readme(subject_id)
            df = pd_old.DataFrame(readme_dict, index=[subject_id])
            dframes.append(df)

        df = pd_old.concat(dframes)
        df.to_csv(self.DATA_PATH + 'readmes.csv')

        
    def merge_with_feature_data(self):
        # Confirm feature files are available
        if os.path.isfile('data/may14_feats4.csv'):
            feat_df = pd.read_csv('data/may14_feats4.csv', index_col=0)
            print(feat_df.info())
        else:
            print('No feature data available. Exiting...')
            return
           
        # Combine data and save
        df = pd.read_csv(f'{self.DATA_PATH}readmes.csv', index_col=0)

        dummy_df = pd_old.get_dummies(df)
        
        dummy_df['subject'] = dummy_df.index.str[1:].astype(int)

        dummy_df = dummy_df[['age', 'height', 'weight', 'gender_ female', 'gender_ male',
                           'coffee_today_YES', 'sport_today_YES', 'smoker_NO', 'smoker_YES',
                           'feel_ill_today_YES', 'subject']]

        merged_df = pd.merge(feat_df, dummy_df, on='subject')

        merged_df.to_csv('data/no_noise.csv')
        

# Data import

In [22]:


#client = Client()

# compute features

# # for ecg
#ddf['ECG'] 
# # for eda

# # for emg

# # for resp

# # for temp

# # for acc

# # for bvp




# correlation matrix
#correlation_value = ddf['column1'].corr(ddf['column2']).compute()



# Initializing the client



In [23]:
def get_ecg_nans():
    x = 0
    return {'bpm': np.nan , 'ibi': np.nan, 'sdnn': np.nan, 'sdsd': np.nan, 
            'rmssd': np.nan, 'pnn20': np.nan, 'pnn50': np.nan}

def get_net_accel(data):
    accn_e4 = (data['ACCx_hx'] ** 2 + data['ACCy_hx'] ** 2 + data['ACCz_hx'] ** 2).apply(lambda x: np.sqrt(x))
    accn_hx = (data['ACCx_hx'] ** 2 + data['ACCy_hx'] ** 2 + data['ACCz_hx'] ** 2).apply(lambda x: np.sqrt(x))
    acc = {'hx': accn_hx, 'e4':accn_e4}
    return acc


In [24]:
import random
from fractions import Fraction
import dask.dataframe as dd
from dask.distributed import Client

import pandas as pd
import heartpy as hp
import numpy as np

Path = '/projects/bbnp/Vansh/data/RADWear/'

ddf = dd.read_parquet(Path+'combined_participants.parquet/*.parquet', engine='pyarrow')
#display(ddf)
pdf = ddf.compute()
#display(pdf)

subject_ids = [7, 9, 12, 14, 16, 17, 21]
n_rand_samples = 10
list(pdf.index)[-1]
m2df = pd.DataFrame()
feat_df = pdf.copy(deep=True)
feat_df.columns[0:13]
feat_df = feat_df.drop(feat_df.columns[0:13], axis=1)
ecg_columns = ['bpm', 'ibi', 'sdnn', 'sdsd',
               'rmssd', 'pnn20', 'pnn50']
m2df, m4df = pd.DataFrame(), pd.DataFrame()
for i in ecg_columns:
    m2df['ECG_'+i], m4df['ECG_'+i]= [], []
m1 = {}
m2 = {}
fs_re = 64
    
def get_more_stats(df, sig_name):
    ops_list = ['max', 'min','mean','corr', 'abs', 'cumsum', 'kurt', 'skew']
    f_df = pd.DataFrame()
    for op in ops_list:
        f_df[sig_name+'_max'] = df[sig_name].max()
        f_df[sig_name+'_min'] = df[sig_name].min()
        f_df[sig_name+'_mean'] = df[sig_name].mean()
        f_df[sig_name+'corr'] = df[sig_name].mean() #df.corr()
        f_df[sig_name+'abs'] = df[sig_name].abs()
        f_df[sig_name+'cumsum'] = df[sig_name].cumsum()
        f_df[sig_name+'kurt'] = df[sig_name].kurt()
        f_df[sig_name+'skew'] = df[sig_name].skew()
        f_df[sig_name+'std'] = df[sig_name].std()
    return f_df

        


In [25]:
seg_f_df = pd.DataFrame()
for p in subject_ids[2:4]:
    #pdf[pdf['participant']==p]
    #print('display pd for p,', p)
    #display(pdf[pdf['participant']==p].head(2))
    p_pdf = pdf[pdf['participant']==p]
    win_ind = random.sample(range(0,len(p_pdf),16), n_rand_samples)

    print('participant p', p,'has n = ', len(p_pdf),' data points')
    
    for i in win_ind:
        
        print(f'progress: {win_ind.index(i)/n_rand_samples*100:.1f} %')
        print
        seg_df = p_pdf[i:3840+i]
        seg_len = len(seg_df)
        # computing ECG features
        ''' 'ECG':['meanHR', 'stdHR', 'meanHRV', 'stdHRV', 'rmsHRV', 'NN50', 'pNN50', 'lf/hf', 'lfnorm', 'hfnorm'],'''
        try: 
            _, m2 = hp.process(seg_df['ECG'].values, 64)
        except:
            _, m2 = np.nan, get_ecg_nans()

        #seg_f_df = pd.concat([seg_f_df, m2])
        
        for key in m2.keys():
            seg_f_df['ECG_'+key] = m2[key]
        del m2

        seg_f_df['ECG_peak_freq'] = get_peak_freq(seg_df['ECG'])
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'ECG_bpm')],axis=1) 


        # computing BVP features
        ''' 'BVP':['meanHR', 'stdHR', 'meanHRV', 'stdHRV', 'rmsHRV', 'NN50', 'pNN50', 'lf/hf', 'lfnorm', 'hfnorm']}'''
        try: 
            _, m4 = hp.process(seg_df['BVP'].values, 64)
        except:
            _, m4 = np.nan, get_ecg_nans()
        #seg_f_df = pd.concat([seg_f_df, m4])
        for key in m4.keys():
            seg_f_df['BVP_'+key] = m4[key]
        del m4

        seg_f_df['BVP_peak_freq'] = get_peak_freq(seg_df['BVP'])
        
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'BVP_bpm')],axis=1) 

        '''    
        hp.process(data['ECG'].dropna().reset_index(drop=True), 700)
        return {'bpm': m['bpm'], 'ibi': m['ibi'], 'sdnn': m['sdnn'], 'sdsd': m['sdsd'], 
                'rmssd': m['rmssd'], 'pnn20': m['pnn20'], 'pnn50': m['pnn50']}
        ''' 
        
        # # for eda 
        ''''EDA':['min', 'max', 'mean', 'std', 'mean_phasic', 'std_phasic', 'mean_tonic', 'std_tonic', 'corr(scl, t)', 'scr_max_rise', 'scr_avg_recovery', 'scr_max_recovery', 'scr_avg_halfrecovery', 'scr_max_halfrecovery', 'scr_avg_scl', 'scr_max_scl', 'scr_avg_phasic', 'scr_max_phasic', 'scr_avg_tonic', 'scr_max_tonic'],'''
        seg_df['EDA'].values
        seg_f_df['EDA_slope'] = get_slope(seg_df['EDA'])
        seg_f_df['EDA_drange'] = get_dynamic_range(seg_df['EDA'])
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'EDA')],axis=1) 
        
        eda_data = nk.eda_phasic(nk.standardize(seg_df['EDA']), sampling_rate=fs_re)
        seg_f_df['EDA_SCR'] = eda_data['EDA_Phasic']
        seg_f_df['EDA_SCL'] = eda_data['EDA_Tonic']
        
        
        #################### RESP IS MISSING FOR NOW, WILL PROCESS LATER ############################
        # # for RESP
        #''''RESP':['br', 'mean_inhal', 'std_inhal', 'mean_exhal', 'std_exhal', 'mean_ie', 'std_ie', 'mean_inhal_duration', 'std_inhal_duration', 'mean_exhal_duration', 'std_exhal_duration', 'mean_inhal_exhal_duration', 'std_inhal_exhal_duration'], '''
        #seg_df['RESP_rate'], seg_df['RESP_Inhal_mean'], seg_df['RESP_Inhal_std'], seg_df['RESP_Exhal_mean'], seg_df['RESP_Exhal_std'], seg_df['RESP_I/E'] = get_resp_features(seg_df['RESP'])
        #feat_df['RESP_rate'], feat_df['RESP_Inhal_mean'], feat_df['RESP_Inhal_std'], feat_df['RESP_Exhal_mean'], feat_df['RESP_Exhal_std'], feat_df['RESP_I/E'] = get_resp_features(seg_df['RESP'])
        #if len(seg_len) > 0:
        #    seg_f_df['RESP_rate'], seg_f_df['RESP_Inhal_mean'], seg_f_df['RESP_Inhal_std'], seg_f_df['RESP_Exhal_mean'], seg_f_df['RESP_Exhal_std'], seg_f_df['RESP_I/E'] = get_resp_features(seg_df['RESP'])
        seg_f_df['BR'] = seg_df['BR']
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'BR')],axis=1) 

        # # for temp
        ''''TEMP':['mean', 'std', 'mean_slope', 'std_slope', 'mean_drange', 'std_drange', 'min', 'max', 'slope', 'drange']'''
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'TEMP')],axis=1) 
        seg_f_df['TEMP_drange'] = get_dynamic_range(seg_df['TEMP'])
        seg_f_df['TEMP_slope'] = get_slope(seg_df['TEMP'])
        
        
        # # for acc
        # # # acc hx
        
        seg_f_df['ACCx_hx_abs_integral'] = get_absolute_integral(seg_df['ACCx_hx'])
        seg_f_df['ACCy_hx_abs_integral'] = get_absolute_integral(seg_df['ACCy_hx'])
        seg_f_df['ACCz_hx_abs_integral'] = get_absolute_integral(seg_df['ACCz_hx'])
        seg_f_df['ACCn_hx_abs_integral'] = get_absolute_integral(get_net_accel(seg_df)['hx'])
        seg_f_df['ACCx_hx_peak_freq'] = get_peak_freq(seg_df['ACCx_hx'])
        seg_f_df['ACCy_hx_peak_freq'] = get_peak_freq(seg_df['ACCy_hx'])
        seg_f_df['ACCz_hx_peak_freq'] = get_peak_freq(seg_df['ACCz_hx'])
        
        seg_f_df['ACCx_hx_mean'] = seg_df['ACCx_hx'].mean()
        seg_f_df['ACCy_hx_mean'] = seg_df['ACCy_hx'].mean()
        seg_f_df['ACCz_hx_mean'] = seg_df['ACCz_hx'].mean()
        seg_f_df['ACCn_hx_mean'] = get_net_accel(seg_df)['hx'].mean()
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'ACCx_hx')],axis=1) 
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'ACCy_hx')],axis=1) 
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'ACCz_hx')],axis=1) 
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'ACCn_hx')],axis=1) 

        
        # # # acc e4
        seg_f_df['ACCx_e4_abs_integral'] = get_absolute_integral(seg_df['ACCx_e4'])
        seg_f_df['ACCy_e4_abs_integral'] = get_absolute_integral(seg_df['ACCy_e4'])
        seg_f_df['ACCz_e4_abs_integral'] = get_absolute_integral(seg_df['ACCz_e4'])
        seg_f_df['ACCn_e4_abs_integral'] = get_absolute_integral(get_net_accel(seg_df)['e4'])
        seg_f_df['ACCx_e4_peak_freq'] = None if seg_len == 0 else get_peak_freq(seg_df['ACCx_e4'])
        seg_f_df['ACCy_e4_peak_freq'] = None if seg_len == 0 else get_peak_freq(seg_df['ACCy_e4'])
        seg_f_df['ACCz_e4_peak_freq'] = None if seg_len == 0 else get_peak_freq(seg_df['ACCz_e4'])
        
        seg_f_df['ACCx_e4_mean'] = seg_df['ACCx_e4'].mean()
        seg_f_df['ACCy_e4_mean'] = seg_df['ACCy_e4'].mean()
        seg_f_df['ACCz_e4_mean'] = seg_df['ACCz_e4'].mean()
        seg_f_df['ACCn_e4_mean'] = get_net_accel(seg_df)['e4'].mean()        
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'ACCx_e4')],axis=1) 
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'ACCy_e4')],axis=1) 
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'ACCz_e4')],axis=1) 
        seg_f_df = pd.concat([seg_f_df, get_more_stats(seg_df,'ACCn_e4')],axis=1) 

        ''''ACC':['mean', 'std', 'abs integral',' mean_slope', 'std_slope', 'mean_drange', 'std_drange', 'min', 'max', 'slope', 'drange', 'peak frequency'], # for x, y, z, and norm'''
  
        
        
        

        feat_df.loc[i] = seg_f_df

'''
features_list =  'TEMP':['mean', 'std', 'mean_slope', 'std_slope', 'mean_drange', 'std_drange', 'min', 'max', 'slope', 'drange'],
                 'ACC':['mean', 'std', 'abs integral',' mean_slope', 'std_slope', 'mean_drange', 'std_drange', 'min', 'max', 'slope', 'drange', 'peak frequency'], # for x, y, z, and norm
             
'BVP', 'ECG', 'EDA',
'TEMP', 'BR',
'ACCx_hx', 'ACCy_hx', 'ACCz_hx', 
'ACCx_e4', 'ACCy_e4', 'ACCz_e4',
'''


IndentationError: unexpected indent (1643587067.py, line 51)

In [ ]:
seg_df

In [ ]:
seg_df.drop('daily_check_in_date',axis=1)

In [ ]:
display(type(seg_df))
#display(seg_df)
#print('seg_df mean: ',seg_df.mean())
#display('seg_df types',seg_df.dtypes)  
display(seg_df.drop('daily_check_in_date',axis=1))
display('seg_df types',seg_df.drop('daily_check_in_date',axis=1).dtypes)  
display(type(love))
love = seg_df.drop('daily_check_in_date',axis=1)
seg_df.max(), seg_df.min(), seg_df.mean()


In [ ]:
(p_pdf['ECG'][i:3840+i].index-97177717)


In [ ]:
p_pdf['ECG'][i:3840+i]

plt.figure();
plt.plot((p_pdf['ECG'][i:3840+i].index-97177717), p_pdf['ECG'][i:3840+i].values);
plt.title('ECG', fontsize=15)
plt.xlabel('index');
plt.xticks(fontsize=10);
plt.yticks(fontsize=10);
#plt.xscale('log');
plt.ylabel('Accuracy', fontsize=20);


_, m2 = hp.process(p_pdf['ECG'][i:3840+i].values, sample_rate=64, windowsize= 60)

In [ ]:
ddf = ddf.repartition(npartitions=150)
print(ddf.columns)
display(ddf)

EDA          
EDA_SCR      
EDA_SCL      
BVP          
TEMP         
ACC_x        
ACC_y        
ACC_z        
label        
ECG          
EDA_C        
EDA_SCR_C    
EDA_SCL_C    
ACC_x_C      
ACC_y_C      
ACC_z_C      
EMG_C        
Resp_C       
Temp_C 

In [ ]:
features_list = {'ECG':['meanHR', 'stdHR', 'meanHRV', 'stdHRV', 'rmsHRV', 'NN50', 'pNN50', 'lf/hf', 'lfnorm', 'hfnorm'],
                 'EDA':['min', 'max', 'mean', 'std', 'mean_phasic', 'std_phasic', 'mean_tonic', 'std_tonic', 'corr(scl, t)', 'scr_max_rise', 'scr_avg_recovery', 'scr_max_recovery', 'scr_avg_halfrecovery', 'scr_max_halfrecovery', 'scr_avg_scl', 'scr_max_scl', 'scr_avg_phasic', 'scr_max_phasic', 'scr_avg_tonic', 'scr_max_tonic'],
                 'RESP':['br', 'mean_inhal', 'std_inhal', 'mean_exhal', 'std_exhal', 'mean_ie', 'std_ie', 'mean_inhal_duration', 'std_inhal_duration', 'mean_exhal_duration', 'std_exhal_duration', 'mean_inhal_exhal_duration', 'std_inhal_exhal_duration'],
                 'Temp':['mean', 'std', 'mean_slope', 'std_slope', 'mean_drange', 'std_drange', 'min', 'max', 'slope', 'drange'],
                 'ACC':['mean', 'std', 'abs integral',' mean_slope', 'std_slope', 'mean_drange', 'std_drange', 'min', 'max', 'slope', 'drange', 'peak frequency'], # for x, y, z, and norm
                 'BVP':['meanHR', 'stdHR', 'meanHRV', 'stdHRV', 'rmsHRV', 'NN50', 'pNN50', 'lf/hf', 'lfnorm', 'hfnorm']}


## ECG features

In [ ]:
len(ddf.partitions[0]['ECG'])

In [ ]:
ddf.partitions[0]['ECG'].compute()
a = 0

In [ ]:
ddf.partitions[i]['ECG'].compute()[0:3840]

In [ ]:
i = 0
for j in range(561,540,-1):
    #print('[i*16:3840+16*i]:  [', j*16,', ', 3840+16*j,']')
    
    ddf.partitions[i]['ECG'].compute()[j*16:3840+16*j]
    
    _, m2 = hp.process(ddf.partitions[i]['ECG'].compute()[start:end].values, sample_rate=64, windowsize= 60)
    m2df = pd.DataFrame([m2])
    del m2
    m2ddf = dd.from_pandas(m2df, npartitions=100)
    del m2df
    m2dddf = dd.multi.concat([m2ddf, m2ddf])
    print(j)

In [ ]:
m2

In [ ]:

hp_results = dd.from_pandas(pd.DataFrame(m2), npartitions=150)

In [ ]:
dd.concat([ddf,ddf_i],axis=0)


In [ ]:
wd2

In [ ]:
print(type(wd2))
print(type(m2))

In [ ]:
ddf.partitions[0].rolling(window=64, min_periods=1).mean().compute()

In [ ]:
sample_rate = 64
wd200, m200 = hp.process(ddf.partitions[0]['ECG'].compute(), sample_rate=64,calc_freq = True)
n_window = WINDOW_IN_SECONDS*sample_rate 

In [ ]:
wd2['hr']

In [ ]:
for i in range(150):
    print(len(ddf.partitions[i]))
#ddf.partitions[0].iloc[:, len_ddf_part//100]


## EDA features

In [ ]:
eda_data = nk.eda_phasic(nk.standardize(eda_df['EDA']), sampling_rate=fs_dict['EDA'])
eda_df['EDA_SCR'] = eda_data['EDA_Phasic']
eda_df['EDA_SCL'] = eda_data['EDA_Tonic']

## BVP features

In [ ]:
eda_data = nk.eda_phasic(nk.standardize(eda_df['EDA']), sampling_rate=fs_dict['EDA'])
    eda_df['EDA_SCR'] = eda_data['EDA_Phasic']
    eda_df['EDA_SCL'] = eda_data['EDA_Tonic']

## RESP features

## TEMP features

## ACC features

In [ ]:
ddf.partitions[0]

In [ ]:
def get_window_stats_ecg(data, label=-1):
    wd, m = hp.process(data['ECG'].dropna().reset_index(drop=True), 64)
    return {'bpm': m['bpm'], 'ibi': m['ibi'], 'sdnn': m['sdnn'], 'sdsd': m['sdsd'], 
            'rmssd': m['rmssd'], 'pnn20': m['pnn20'], 'pnn50': m['pnn50']}

In [ ]:
p = subject_ids[5]

print(Path +  '/p_'+str(p)+'.pkl: ', os.path.isfile(Path +  '/p_'+str(p)+'.pkl'))
#with open(Path +  '/p_'+str(p)+'.pkl', 'rb') as f:
#    df = pickle.load(f)
#df = pd_old.read_pickle('data/RADWear/p_'+str(p)+'.pkl')


## Correlation of Each Feature to Target
#vals = abs(df.corr()['label']).sort_values(ascending=False)
#for i in range(len(vals)):
#    print(vals.index[i], vals[i])

In [ ]:
subject_ids[1:2]

In [ ]:
print(pd.__version__)

In [ ]:
df.keys() 

In [ ]:
features = df.loc[:, df.columns != 'label'].columns

In [ ]:
for ft_idx in range(len(features)):
    print(features[ft_idx], ft_idx)

In [ ]:
X = df.drop('label', axis=1).values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

# Modeling

## Cross Validation

### K-Fold Cross Validation

### Leave-One-Out Cross Validation

## Linear Discriminant Analysis

In [ ]:
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  


lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred = lda.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)  
print(cm)  
print('Accuracy: ' + str(accuracy_score(y_test, y_pred)))  
lda_baseline_acc = accuracy_score(y_test, y_pred)

## XG Boost

In [ ]:
ests = np.geomspace(1, 1000, 100)
xg_accs = []
for est in ests:
    xg = XGBClassifier(n_estimators=int(est), n_jobs=-1)
    xg.fit(X_train, y_train)
    y_pred = xg.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    xg_baseline_acc = accuracy_score(y_test, y_pred)
    xg_accs.append(xg_baseline_acc)

In [ ]:
plt.figure(figsize=(12,12));
plt.plot(ests, xg_accs);
plt.title('XGB Accuracy vs. Num Estimators', fontsize=25)
plt.xlabel('Number Estimators', fontsize=20);
plt.xticks(fontsize=15);
plt.yticks(fontsize=15);
plt.xscale('log');
plt.ylabel('Accuracy', fontsize=20);

In [ ]:
xg = XGBClassifier(n_estimators=100, n_jobs=-1)
xg.fit(X_train, y_train)
y_pred = xg.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
xg_baseline_acc = accuracy_score(y_test, y_pred)
xg_accs.append(xg_baseline_acc)
importances = xg.feature_importances_

xgb_importances = pd.Series(importances, index=features).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(12, 5))
xgb_importances[:20].plot.barh(ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

## Random Forest Classifier

In [ ]:
classifier = RandomForestClassifier(random_state=0)
classifier.fit(X_train, y_train)  
y_pred = classifier.predict(X_test)  

In [ ]:
cm = confusion_matrix(y_test, y_pred)  
print(cm)  
print('Accuracy: ' + str(accuracy_score(y_test, y_pred)))  
rf_baseline_acc = accuracy_score(y_test, y_pred)

### Feature Importance (Top 20)

In [ ]:
importances = classifier.feature_importances_
forest_importances = pd.Series(importances, index=features).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(12, 5))
forest_importances[:20].plot.barh(ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

## Support Vector Machine

### Linear SVM

In [ ]:
# Create a SVC classifier using a linear kernel
clf = SVC(kernel='linear', C=1, random_state=0)
# Train the classifier
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_out = clf.predict(X_test)
lm_svc=(classification_report(y_test, y_out, digits=4))
print(lm_svc)
svm_baseline_acc = accuracy_score(y_test, y_pred)

### Feature Importance (Top 20)

In [ ]:
pd.Series(abs(clf.coef_[0]), index=features).nlargest(10).plot(kind='barh')

### Linear SVM (lambda 2)

In [ ]:
# Create a SVC classifier using a linear kernel
clf = SVC(kernel='linear', C=0.9, random_state=0)
# Train the classifier
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_out = clf.predict(X_test)
lm_svc=(classification_report(y_test, y_out, digits=4))
print(lm_svc)
svm_baseline_acc = accuracy_score(y_test, y_pred)

In [ ]:
pd.Series(abs(clf.coef_[0]), index=features).nlargest(10).plot(kind='barh')

## Single Decision Tree Feature Importance

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X, y)

feat_importance = dt.feature_importances_
plt.figure(figsize=(40,20));
tree.plot_tree(dt,fontsize=10, feature_names=features);

### Top 5 Most Improtant Features

In [ ]:
sorted(list(zip(feat_importance, features)), reverse=True)[:5]

# Adding Noise

## Signal to Noise Ratio

For a non-constant signal $S$ and noise $N$, the signal to noise ratio is defined as the following:
$$ SNR = \frac{\mathbb{E}[S^2]}{\mathbb{E}[N^2]} $$

The expected value $\mathbb{E}[X]$ of any continuous random variable $X$ is $\int_{-\infty}^{\infty} x p(x) dx $, where $p(x)$ is its associated probability density function.

For homoskedastic noise, we can use closed form expressions to compute $E[N^2]$.

- For Gaussian distributed noise $N$ ~ $n(\mu, \sigma^2)$, notice that $\text{V}[N] = \mathbb{E}[N^2] - (\mathbb{E}[N])^2,$ so $\mathbb{E}[N^2] = \text{V}[N] + (\mathbb{E}[N])^2 = \sigma^2 + \mu$. In our case $\mu = 0$, so $\mathbb{E}[N^2] = \sigma^2$.

- For uniformly distributed noise $N$ ~ $u(\alpha, \beta)$, by the same logic as above $\mathbb{E}[N^2] = \left(\frac{\alpha - \beta}{2}\right)^2$.

- For frequency-domain noise $N$ of the form $A\sin(2\pi x \frac{1}{f}) + y, \mathbb{E}[N^2] \approx y^2 + \frac{A^2}{2}$. Note the $\approx$ since we cannot guarantee that the signal will end precisely on the end of the sin wave.

For heteroskedastic noise, because there is no closed form expression, we simply take `N.mean()` where $N$ is our noise

In [ ]:
def signal_to_noise_ratio(signal, noise_type, noise_dist, noise=None, sigma=None, alpha=None, beta=None, 
                          vertical_shift=None, amplitude=None):
    """
    Function: Computes the signal to noise ratio for a given signal and its corresponding noise.
    
    :param:
        signal (array or ndarray): The signal we are evaluating
        noise_type (string): 'Heteroskedastic' or 'Homoskedastic'
        noise_dist (string): 'Uniform', 'Gaussian', or 'Frequency' for now
        noise (array or ndarray): Only passed in if we have heteroskedastic noise
        sigma (float): Sigma parameter of the gaussian
        alpha (float): Alpha parameter of the uniform
        beta (float): Beta parameter of the uniform
        vertical_shift (float): Vertical shift parameter of the frequency
        amplitude (float): Amplitude parameter of the frequency
        
    :return
        signal_to_noise_ratio (float): Signal to noise ratio... E[S^2]/E[N^2]
    """
    
    # Calculate E[S^2]
    e_s2 = (signal**2).mean()
    e_n2 = None
    
    if noise_type == 'Homoskedastic':
        # Calculate E[N^2] for the pertinent case
        if noise_dist == 'Uniform':
            e_n2 = (0.5*(alpha - beta))**2
        elif noise_dist == 'Gaussian':
            e_n2 = sigma**2
        elif noise_dist == 'Frequency':
            e_n2 = vertical_shift**2 + (amplitude**2)/2
    elif noise_type == 'Heteroskedastic':
        e_n2 = (noise**2).mean()
    
    # Return the signal to noise ratio
    return e_s2/e_n2

## Calculate Distribution Parameters from SNR

Given a signal $S$, we can specify a signal to noise ratio $SNR = \frac{\mathbb{E}[S^2]}{\mathbb{E}[N^2]}$ and use this to calculate $\mathbb{E}[N^2]$ because $SNR$ and $\mathbb{E}[S^2]$ are known. So $\mathbb{E}[N^2] = \frac{\mathbb{E}[S^2]}{SNR}$.

Then, for any homoskedastic noise following a well-defined probability density function (PDF), we can solve for the parameters of the PDF using the known value $\mathbb{E}[N^2]$.

- For Gaussian distributed noise $N$ ~ $n(\mu, \sigma^2)$, notice that $\text{V}[N] = \mathbb{E}[N^2] - (\mathbb{E}[N])^2,$ so $\mathbb{E}[N^2] = \text{V}[N] + (\mathbb{E}[N])^2 = \sigma^2 + \mu$. In our case $\mu = 0$, so $\mathbb{E}[N^2] = \sigma^2$. Thus, $\sigma^2 = \frac{\mathbb{E}[S^2]}{SNR}$.

In [ ]:
def calculate_param(signal, noise_type, signal_to_noise_ratio):
    """
    Function: Calculates the parameters of a continuous probability density function given
    our desired signal to noise ratio and signal.
    
    :param:
        signal (array or ndarray): Our signal
        noise_type (string): Probability distribution of our noise (i.e., Gaussian)
        signal_to_noise_ratio (float): Desired signal to noise ratio
    
    :return
    (for now, just)
        sigma (float): sigma of the gaussian
    """
    
    return (signal**2).mean()/signal_to_noise_ratio

## Gaussian Noise

The Gaussian probability density function is of the following form:
\begin{equation}\label{eq:}
f(x) = \frac{1}{\sigma \sqrt{2 \pi}}exp\left(-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2\right)
\end{equation}

### Estimating $\mu$ and $\sigma$ of the Gaussian

#### Greatest $n$-Differential with Homoskedasticity Approach

For a signal $S$, the greatest $n$-differential with homoskedasticity approach constructs a Gaussian distribution such that $\mu$ = 0 and $\sigma = \alpha \cdot max(|S_i - S_{i+n}|)$, where $max(|S_i - S_{i+n}|)$ denotes the maximum absolute difference of the signal between index $i$ and $i+n$ in the entire signal, and $\alpha$ is a parameter that multiplicatively scales the intensity of the added noise. We can choose to set $n$ to any value, although we have empirically found $n = 5$ to be the best. We set $\mu$ to $0$ so we don't vertically shift the original signal after adding noise. 

In conclusion, we randomly sample from the following probability density function:
$$
f(x) = \frac{1}{\alpha \cdot max(|S_i - S_{i+n}|) \sqrt{2 \pi}}exp\left(-\frac{1}{2}\left(\frac{x}{\alpha \cdot max(|S_i - S_{i+n}|)}\right)^2\right)
$$

This noise exhbits homoskedasticity because it does not vary with time.

In [ ]:
def gaussian_homoskedastic(signal_name, signal, signal_to_noise_ratio=None):
    """
    Constructs a homoskedastic gaussian probability density function, samples noise from it,
    then adds noise to the signal
    
    :param:
        signal_name (string): The name of the signal (i.e., ECG)
        signal (array or ndarray): The signal we wish to add noise to
        signal_to_noise_ratio (float): [default: None] If specified, our desired SNR.
        
    :return
        noisy_signal: The signal after we have added noise to it
    """
    
    x_new = None

    if signal_name == 'ACC':
        alpha = 0.5
        mu = 0
        # Noise X Axis
        x_axis = signal[:,0]
        sigma = calculate_param(x_axis, 'Gaussian', signal_to_noise_ratio)
        s = np.random.normal(mu, sigma, 1000)
        x_axis_new = np.copy(x_axis)
        for i in range(len(x_axis_new)):
            x_axis_new[i] += float(np.random.normal(mu, sigma, 1))
        # Noise Y Axis
        y_axis = signal[:,1]
        sigma = calculate_param(y_axis, 'Gaussian', signal_to_noise_ratio)
        s = np.random.normal(mu, sigma, 1000)
        y_axis_new = np.copy(y_axis)
        for i in range(len(y_axis_new)):
            y_axis_new[i] += float(np.random.normal(mu, sigma, 1))
        # Noise Z Axis
        z_axis = signal[:,2]
        sigma = calculate_param(z_axis, 'Gaussian', signal_to_noise_ratio)
        s = np.random.normal(mu, sigma, 1000)
        z_axis_new = np.copy(z_axis)
        for i in range(len(z_axis_new)):
            z_axis_new[i] += float(np.random.normal(mu, sigma, 1))

        # Put together noisy signal
        x_new = np.zeros((len(signal), 3))
        x_new[:,0] = x_axis_new
        x_new[:,1] = y_axis_new
        x_new[:,2] = z_axis_new

        return (x_new, sigma)
    else: 
        # Store original shape
        original_shape = signal.shape

        # Caveat: some signals like ACC have three axes
        # Flatten signal to be 1d
        x = np.ravel(signal)

        # Calculate mean and Standard deviation
        alpha = 0.5
        mu = 0
        sigma = calculate_param(x, 'Gaussian', signal_to_noise_ratio)

        # test that this works
        x_new = x + np.random.normal(mu, sigma, (len(x),))

#         for i in range(len(x_new)):
#             x_new[i] += float(np.random.normal(mu, sigma, 1))

        return (np.array(x_new).reshape(original_shape), sigma)

#### Greatest $n$-Differential Approach with Heteroskedasticity

For a signal $S$, the greatest $n$-differential with heteroskedasticity approach constructs a Gaussian probability density function such that $\mu$ = 0 and $\sigma = \alpha \cdot max(|S_{t-100\thinspace\leq\thinspace i \thinspace\leq\thinspace t} - S_{t-100\thinspace\leq\thinspace i+n \thinspace\leq\thinspace t}|)$, where $max(|S_{t-100\thinspace\leq\thinspace i \thinspace\leq\thinspace t} - S_{t-100\thinspace\leq\thinspace i+n \thinspace\leq\thinspace t}|)$ denotes the maximum absolute difference of the signal between any index $i$ and $i+n$ in the last $100$ samples from index $t$, and $\alpha$ is a parameter that multiplicatively scales the intensity of the added noise. We can choose to set $n$ to any value, although we have empirically found $n = 5$ to be the best. We set $\mu$ to $0$ so we don't vertically shift the original signal after adding noise. 

At each fixed time value $t \in [S_{start}, S_{end}]$, we randomly sample from the following probability density function:
$$
\epsilon_t = f(x, t) = \frac{1}{\alpha \cdot max(|S_{t-100\thinspace\leq\thinspace i \thinspace\leq\thinspace t} - S_{t-100\thinspace\leq\thinspace i+n \thinspace\leq\thinspace t}|) \sqrt{2 \pi}}exp\left(-\frac{1}{2}\left(\frac{x}{\alpha \cdot max(|S_{t-100\thinspace\leq\thinspace i \thinspace\leq\thinspace t} - S_{t-100\thinspace\leq\thinspace i+n \thinspace\leq\thinspace t}|)}\right)^2\right)
$$

We add each $\epsilon_t$ to its respective value of $S_t$ to add noise to the signal. 

In [ ]:
def gaussian_heteroskedastic(signal):
    """
    Constructs a heteroskedastic gaussian probability density function, samples noise from it,
    then adds noise to the signal
    
    :param:
        signal (array or ndarray): The signal we wish to add noise to
        
    :return
        noisy_signal: The signal after we have added noise to it
    """
    
    # Store original shape
    original_shape = signal.shape
    
    # Caveat: some signals like ACC have three axes
    # Flatten signal to be 1d
    x = np.ravel(signal)
    
    # Calculate mean and Standard deviation
    alpha = 0.5
    mu = 0
    sigma = None
    
    # Add heteroskedastic noise
    x_new = np.copy(x)
    noise = np.array([])
    window_len = 100
    for i in range(100, len(x)):
        x_rolling = x[i-100:i]
        sigma = alpha*abs(x_rolling - np.roll(x_rolling, 5)).max()
        noise_i = float(np.random.normal(mu, sigma, 1))
        np.append(noise, noise_i)
        x_new[i] += noise_i
    for i in range(len(x)-100, len(x)):
        x_new[len(x)-i] += float(np.random.normal(mu, sigma, 1))  

    return (np.array(x_new).reshape(original_shape), signal_to_noise_ratio(signal=x, noise=noise, noise_type='Heteroskedastic', 
                                        noise_dist='Gaussian'))

## Frequency Domain Noise

A sinusoidal wave is of the form:

\begin{equation}\label{eq:1}
f(x) = Asin(2 \pi T*x) + y
\end{equation}

where $A$ is the amplitude of the wave, $T = \frac{1}{f}$ is the period, and $y$ is the vertical shift. Note that we have omitted the phase shift parameter since we will overlay the noise to the data at pre-determined locations.

In [ ]:
# Constructs a sinusoidal wave
def sinusoidal_wave(amplitude, frequency, vertical_shift, x):
    return amplitude * np.sin(2*np.pi*(1/frequency)*x) + vertical_shift

Each physiological signal has a different sampling frequency.

In [ ]:
fs_dict = {'ACC': 64, 'BVP': 64, 'EDA': 64, 'TEMP': 64, 'label': 64, 'Resp': 64, 'ECG': 64, 'chest': 64}

In [ ]:
def frequency(signal, hz):
    """
    Constructs a heteroskedastic gaussian probability density function, samples noise from it,
    then adds noise to the signal
    
    :param:
        signal (array or ndarray): The signal we wish to add noise to
        hz (float): The hertz (frequency) at which we wish to add noise)
        
    :return
        noisy_signal: The signal after we have added noise to it
    """
    
    # Store original shape
    original_shape = signal.shape
    
    # Caveat: some signals like ACC have three axes
    # Flatten signal to be 1d
    x = np.ravel(signal)
    
    # Caveat: each signal has a different base sampling frequency
    # Add frequency noise
    sinusoidal_wave(amplitude, 1/hz, vertical_shift, x)
    x_new = np.copy(x)

    return np.array(x_new).reshape(original_shape)

### Estimating $A$, $T$ and $y$ of the Sinusoidal

## Exporting Noisy Files

### Function to Write File

In [ ]:
def write_file(path, file_name, data):
    """
    Function: Writes filename to its specified path and then dumps data in .pkl format to that file.
    
    :param:
        path (string): Where you want to write (create) the file
        file_name (string): What you want to name the file you're creating
        data (array or ndarray or pd.DataFrame): What you want inside the file you're creating
    
    """
    
    filename = file_name + '.pkl'
    with open(os.path.join(path, filename), 'wb') as dest:
        pickle.dump(data, dest)

Root Directory (you will likely need to change, unless you are Sam)

In [ ]:
rootdir = r'C:\Users\Vansh\Desktop\anxietyFB-1\data\WESAD'

### Function to Add Noise

In [ ]:
#snrs = [0.01, 0.05, 0.1, 0.2, 0.5]

In [ ]:
snrs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6]

In [ ]:
#snrs = [0.0001, 0.00001]

In [ ]:
def add_noise(rootdir, snrs, body_parts=['wrist', 'chest'],
              signals={'wrist': ['ACC', 'BVP', 'EDA', 'TEMP'],'chest': ['ECG', 'Temp', 'EMG', 'Resp', 'ACC']}, num_runs=10):
    """
    Function: Adds noise to the WESAD data, stored in the specified root directory
    
    :param:
        rootdir (string): The root directory from which to read the WESAD data.
        body_parts (list): [Default: '[wrist', 'chest']] Body parts from which to read the WESAD data
        signals (dict): [Default: see above] The physiological signals each body part has
    :return
        None
    """
    snr=snrs
    patients = []
    patients_with_noise1 = []
    patients_with_noise2 = []
    patients_with_noise3 = []
    patients_with_noise4 = []
    patients_with_noise5 = []
    #patient_idx = 0
    # Iterate through each patient's folder and construct a df with all patient data

    for run in range(num_runs):
        #output_dir = os.path.join(rootdir, f"run_{run+1}")
        #os.makedirs(output_dir, exist_ok=True)
        patient_idx = 0
        for subdir, dirs, files in os.walk(rootdir):
            for file in files:


                # If this is a .pkl file then it's the synchronized features/labels
                # and we want to serialize the file
                if '.pkl' in file and 'gauss' not in file: 
                    output_dir = os.path.join(rootdir, f"{file.split('.')[0]}")
                    output_dir = os.path.join(output_dir, f"run_{run+1}")
                    os.makedirs(output_dir, exist_ok=True)
                    patient = pd.read_pickle(os.path.join(subdir, file))
                    #print(patient)
                    for snr in snrs: 
                        noisy_patient = patient.copy()
                        for body_part in body_parts:
                            # Add noise
                            for sgl in signals[body_part]:
                                # Get signal
                                signal = patient['signal'][body_part][sgl]
                                x_gaussian_homoskedastic, sigma = gaussian_homoskedastic(sgl, signal, snr)
                                noisy_patient['signal'][body_part][sgl] = x_gaussian_homoskedastic

                        #output_subdir = os.path.join(output_dir, f"patient_{patient_idx + 1}")
                        #os.makedirs(output_subdir, exist_ok=True)
                        output_filename = f"{file.split('.')[0]}_gauss_homo_snr_{snr}.pkl"
                        write_file(output_dir, output_filename, noisy_patient)
                        del noisy_patient
                    patient_idx+=1
                    del patient

In [ ]:
num_runs = 10

In [ ]:
snrs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6]
#add_noise(rootdir, snrs, 1)

# Feature Extraction (pt. 2)

Do feature extraction again, this time with the noisy data

In [ ]:
patients_new_noise1 = []
patients_new_noise2 = []
patients_new_noise3 = []
patients_new_noise4 = []
patients_new_noise5 = []
# Iterate through each patient's folder and construct a
# df with all patient data
for subdir, dirs, files in os.walk(rootdir):
    for file in files:

        # If this is a .pkl file then it's the synchronized features/labels
        # and we want to serialize the file
        if '.pkl' in file and 'gauss' in file and 'snr_0.001' in file:
            patients_new_noise1.append(pd.read_pickle(subdir + '/' + file))
        elif '.pkl' in file and 'gauss' in file and 'snr_0.15' in file:
            patients_new_noise2.append(pd.read_pickle(subdir + '/' + file))
        elif '.pkl' in file and 'gauss' in file and 'snr_0.3' in file:
            patients_new_noise3.append(pd.read_pickle(subdir + '/' + file))
        elif '.pkl' in file and 'gauss' in file and 'snr_0.4' in file:
            patients_new_noise4.append(pd.read_pickle(subdir + '/' + file))
        elif '.pkl' in file and 'gauss' in file and 'snr_0.6' in file:
            patients_new_noise5.append(pd.read_pickle(subdir + '/' + file))

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(16,4))
#[4015000:4020000]
axs[0].plot(np.ravel(patients_new_noise1[0]['signal']['chest']['Resp'][3000000 - 100 :3000000]))
axs[0].set_title('SNR: 0.001')
axs[1].plot(np.ravel(patients_new_noise2[0]['signal']['chest']['Resp'][3000000 - 100 :3000000]))
axs[1].set_title('SNR: 0.15')
axs[2].plot(np.ravel(patients_new_noise3[0]['signal']['chest']['Resp'][3000000 - 100 :3000000]))
axs[2].set_title('SNR: 0.3')
axs[3].plot(np.ravel(patients_new_noise4[0]['signal']['chest']['Resp'][3000000 - 100 :3000000]))
axs[3].set_title('SNR: 0.4')
axs[4].plot(np.ravel(patients_new_noise5[0]['signal']['chest']['Resp'][3000000 - 100 :3000000]))
axs[4].set_title('SNR: 0.6')

# Data Preparation (pt. 2)

Prepare the data again, this time with the noisy data

In [ ]:
snrs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6]

In [ ]:
# Class to store the data for each subject
class SubjectDataWithNoise:

    def __init__(self, main_path, subject_number, snr):
        self.name = f'S{subject_number}'
        self.subject_keys = ['signal', 'label', 'subject']
        self.signal_keys = ['chest', 'wrist']
        self.chest_keys = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
        self.wrist_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        with open(os.path.join(main_path, self.name) + '/' + "/run_1/" + self.name + f'_gauss_homo_snr_{snr}.pkl.pkl', 'rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.labels = self.data['label']

    def get_wrist_data(self):
        data = self.data['signal']['wrist']
        data.update({'ACC_C': self.data['signal']['chest']['ACC'],
                     'ECG_C': self.data['signal']['chest']['ECG'],
                     'EDA_C': self.data['signal']['chest']['EDA'],
                     'EMG_C': self.data['signal']['chest']['EMG'],
                     'Resp_C': self.data['signal']['chest']['Resp'],
                     'Temp_C': self.data['signal']['chest']['Temp']
                     })
        return data

    def get_chest_data(self):
        return self.data['signal']['chest']

    def extract_features(self):  # only wrist
        results = \
            {
                key: get_statistics(self.get_wrist_data()[key].flatten(), self.labels, key)
                for key in self.wrist_keys
            }
        return results

In [ ]:
def make_patient_data_with_noise(subject_id):
    global savePath
    global WINDOW_IN_SECONDS

    # Make subject data object for Sx
    for snr in snrs: 
        print(snr)
        subject = SubjectDataWithNoise(main_path='data/WESAD', subject_number=subject_id, snr=snr)

        # Empatica E4 data - now with resp
        e4_data_dict = subject.get_wrist_data()

        # Chest data
        ch_data_dict = subject.get_chest_data()
        
        # norm type
        norm_type = None

        # The 3 classes we are classifying
        grouped, baseline, stress, amusement = compute_features(e4_data_dict, ch_data_dict, subject.labels, norm_type)

        # print(f'Available windows for {subject.name}:')
        n_baseline_wdws = int(len(baseline) / (fs_dict['label'] * WINDOW_IN_SECONDS))
        n_stress_wdws = int(len(stress) / (fs_dict['label'] * WINDOW_IN_SECONDS))
        n_amusement_wdws = int(len(amusement) / (fs_dict['label'] * WINDOW_IN_SECONDS))
        # print(f'Baseline: {n_baseline_wdws}\nStress: {n_stress_wdws}\nAmusement: {n_amusement_wdws}\n')

        #
        baseline_samples = get_samples(baseline, n_baseline_wdws, 1)
        #for col in baseline_samples.columns:
        #    print(col)
        # Downsampling
        # baseline_samples = baseline_samples[::2]
        stress_samples = get_samples(stress, n_stress_wdws, 2)
        amusement_samples = get_samples(amusement, n_amusement_wdws, 0)

        all_samples = pd_old.concat([baseline_samples, stress_samples, amusement_samples])
        all_samples = pd_old.concat([all_samples.drop('label', axis=1), pd_old.get_dummies(all_samples['label'])], axis=1)
        # Save file as csv
        all_samples.to_csv(f'{savePath}{subject_feature_path}/S{subject_id}_feats_with_noise_{snr}.csv')

        subject = None

In [ ]:

def combine_files_with_noise(subjects):
    df_list = []
    for snr in snrs: 
        for s in subjects:
            df = pd.read_csv(f'{savePath}{subject_feature_path}/S{s}_feats_with_noise_{snr}.csv', index_col=0)
            df['subject'] = s
            df_list.append(df)

        df = pd_old.concat(df_list)

        df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str)).apply(lambda x: x.index('1'))
        df.drop(['0', '1', '2'], axis=1, inplace=True)

        df.reset_index(drop=True, inplace=True)

        print('here')
        df.to_csv(f'{savePath}/{snr}_feats4_with_noise.csv')

        counts = df['label'].value_counts()
        print('Number of samples per class:')
        for label, number in zip(counts.index, counts.values):
            print(f'{int_to_label[label]}: {number}')


In [ ]:
class rparser_with_noise:
    # Code adapted from https://github.com/arsen-movsesyan/springboard_WESAD/blob/master/parsers/readme_parser.py
    VALUE_EXTRACT_KEYS = {
        "age": {
            'search_key': 'Age',
            'delimiter': ':'
        },
        "height": {
            'search_key': 'Height',
            'delimiter': ':'
        },
        "weight": {
            'search_key': 'Weight',
            'delimiter': ':'
        },
        "gender": {
            'search_key': 'Gender',
            'delimiter': ':'
        },
        "dominant_hand": {
            'search_key': 'Dominant',
            'delimiter': ':'
        },
        "coffee_today": {
            'search_key': 'Did you drink coffee today',
            'delimiter': '? '
        },
        "coffee_last_hour": {
            'search_key': 'Did you drink coffee within the last hour',
            'delimiter': '? '
        },
        "sport_today": {
            'search_key': 'Did you do any sports today',
            'delimiter': '? '
        },
        "smoker": {
            'search_key': 'Are you a smoker',
            'delimiter': '? '
        },
        "smoke_last_hour": {
            'search_key': 'Did you smoke within the last hour',
            'delimiter': '? '
        },
        "feel_ill_today": {
            'search_key': 'Do you feel ill today',
            'delimiter': '? '
        }
    }
    
    DATA_PATH = 'data/WESAD/'
    parse_file_suffix = '_readme.txt'
    
    
    def __init__(self, snrs, run):
        
        self.readme_locations = {subject_directory: self.DATA_PATH + subject_directory + '/' 
                          for subject_directory in os.listdir(self.DATA_PATH)
                              if re.match('^S[0-9]{1,2}$', subject_directory)}
        
        # Check if parsed readme file is available ( should be as it is saved above )
        if not os.path.isfile('data/readmes.csv'):
            print('Parsing Readme files')
            self.parse_all_readmes()
        else:
            print('Files already parsed.')
            
        self.merge_with_feature_data_with_noise(snrs, run)
        
        
    def parse_readme(self, subject_id):
        with open(self.readme_locations[subject_id] + subject_id + self.parse_file_suffix, 'r') as f:

            x = f.read().split('\n')

        readme_dict = {}

        for item in x:
            for key in self.VALUE_EXTRACT_KEYS.keys():
                search_key = self.VALUE_EXTRACT_KEYS[key]['search_key']
                delimiter = self.VALUE_EXTRACT_KEYS[key]['delimiter']
                if item.startswith(search_key):
                    d, v = item.split(delimiter)
                    readme_dict.update({key: v})
                    break
        return readme_dict


    def parse_all_readmes(self):
        
        dframes = []

        for subject_id, path in self.readme_locations.items():
            readme_dict = self.parse_readme(subject_id)
            df = pd.DataFrame(readme_dict, index=[subject_id])
            dframes.append(df)

        df = pd_old.concat(dframes)
        df.to_csv(self.DATA_PATH + 'readmes.csv')

        
    def merge_with_feature_data_with_noise(self, snrs, run):
        # Confirm feature files are available
        for snr in snrs: 
            if os.path.isfile(f'data/{snr}_feats4_with_noise.csv'):
                feat_df = pd.read_csv(f'data/{snr}_feats4_with_noise.csv', index_col=0)
                print(feat_df.info())
            else:
                print('No feature data available. Exiting...')
                return
            
            # Combine data and save
            df = pd.read_csv(f'{self.DATA_PATH}readmes.csv', index_col=0)

            dummy_df = pd_old.get_dummies(df)
            
            dummy_df['subject'] = dummy_df.index.str[1:].astype(int)

            dummy_df = dummy_df[['age', 'height', 'weight', 'gender_ female', 'gender_ male',
                            'coffee_today_YES', 'sport_today_YES', 'smoker_NO', 'smoker_YES',
                            'feel_ill_today_YES', 'subject']]

            merged_df = pd.merge(feat_df, dummy_df, on='subject')

            merged_df.to_csv(f'data/{run}/noise_snr_{snr}.csv')
        

In [ ]:
import pandas as pd
num_itr = 10
snrs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6]
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
for i in range(6, num_itr):
    if i != 6: 
        add_noise(rootdir, snrs, num_runs=1)
    print(i)
    for patient in subject_ids:
        print(f'Processing data for S{patient}...')
        make_patient_data_with_noise(patient)

    combine_files_with_noise(subject_ids)
    print('Processing complete.')
    rp_with_noise = rparser_with_noise(snrs, i+1)

In [ ]:
#rp_with_noise = rparser_with_noise(snrs, 1)


# Modeling (pt. 2)

Model again, this time with the noisy data

In [ ]:
df = pd.read_csv('data/1/noise_snr_0.6.csv', index_col=0)
pd.set_option('display.max_columns', None) 

In [ ]:
s = df.isna().sum()
for i in range(len(s)):
    if s[i] > 0:
        print(s.index[i], s[i])

In [ ]:
#df = df.loc[:, df.columns != 'Resp_C_rate']
#df
df = df.dropna(axis=1)
df

In [ ]:
features = df.loc[:, df.columns != 'label'].columns

In [ ]:
X = df.drop('label', axis=1).values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

## Linear Discriminant Analysis

In [ ]:
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  


lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred = lda.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)  
print(cm)  
print('Accuracy: ' + str(accuracy_score(y_test, y_pred)))  

## Random Forest Classifier

In [ ]:
classifier = RandomForestClassifier(max_depth=4, random_state=0)
classifier.fit(X_train, y_train)  
y_pred = classifier.predict(X_test)  

In [ ]:
cm = confusion_matrix(y_test, y_pred)  
print(cm)  
print('Accuracy: ' + str(accuracy_score(y_test, y_pred)))  

### Feature Importance (Top 20)

In [ ]:
importances = classifier.feature_importances_
forest_importances = pd.Series(importances, index=features).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(12, 5))
forest_importances[:20].plot.barh(ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

## Support Vector Machine

In [ ]:
# Create a SVC classifier using a linear kernel
clf = SVC(kernel='linear', C=1, random_state=0)
# Train the classifier
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_out = clf.predict(X_test)
lm_svc=(classification_report(y_test, y_out, digits=4))
print(lm_svc)

# Compare Results

Compare the results of the noisy data models and the clean data models

- Plots of SNR (x-axis) vs. accuracy (y-axis)
- Compare feature importances across different noise regimes
    - Develop dynamic evaluation method based on original feature importance / added noise

## Test Each Model Architecture

In [ ]:
snrs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6]
lda_accuracy = {key: [] for key in snrs}
rf_accuracy = {key: [] for key in snrs}
svm_accuracy = {key: [] for key in snrs}
ft_imp_matrix = {key: [] for key in snrs}
ft_imp_schmidt = {key: [] for key in snrs}
# For each signal to noise ratio

for j in range(1, 7):
    for i in range(len(snrs)):
            # Get data
        df = pd.read_csv(f'data/{j}/noise_snr_{snrs[i]}.csv', index_col=0)
            # Since Resp_C_rate is null for the first four, simply get rid of it
        #df = df.loc[:, df.columns != 'Resp_C_rate']
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df = df.dropna(axis=1)
        
            # Get features, label
        X = df.drop('label', axis=1).values
        y = df['label'].values
        
            # Get train test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
            
            # Scale the data
        sc = StandardScaler()  
        X_train = sc.fit_transform(X_train)  
        X_test = sc.transform(X_test)  
            
            # Test LDA
        lda = LinearDiscriminantAnalysis()
        lda.fit(X_train, y_train)
        y_pred = lda.predict(X_test)
        lda_accuracy[snrs[i]].append(accuracy_score(y_test, y_pred))
            
            # Feature importance the same way Schmidt did
        #clf = DecisionTreeClassifier()
        #clf.fit(X, y)

        #feat_importance = clf.tree_.compute_feature_importances(normalize=False)

        #out = io.StringIO()
        #out = export_graphviz(clf, out_file='no_noise.dot')
        #ft_imp_schmidt[snrs[i]].append(sorted(list(zip(feat_importance, features)), reverse=True)[:5])
            
            # Test RF
        classifier = RandomForestClassifier(max_depth=4, random_state=0)
        classifier.fit(X_train, y_train)  
        y_pred = classifier.predict(X_test)  
        rf_accuracy[snrs[i]].append(accuracy_score(y_test, y_pred))
        importances = classifier.feature_importances_
        forest_importances = pd.Series(importances, index=features).sort_values(ascending=False)[:10]
        ft_imp_matrix[snrs[i]].append(forest_importances)

        '''
        fig, ax = plt.subplots(figsize=(12, 5))
        forest_importances[:20].plot.barh(ax=ax)
        ax.set_title("Feature importances using MDI with SNR: " + str(snrs[i]))
        ax.set_ylabel("Mean decrease in impurity")
        fig.tight_layout()
        plt.show();
          '''  
            # Test SVM
        clf = SVC(kernel='linear', C=1, random_state=0)
        clf.fit(X_train, y_train)
        y_out = clf.predict(X_test)    
        svm_accuracy[snrs[i]].append(accuracy_score(y_test, y_pred))

## Plot SNR vs. Accuracy - Multiple

In [ ]:

min_accuracy = min(min(min(lda_accuracy.values())), min(min(rf_accuracy.values())), min(min(svm_accuracy.values()))) - 0.01
max_accuracy = max(lda_baseline_acc, rf_baseline_acc, svm_baseline_acc) + 0.01

q25_lda_accuracy = {key: np.percentile(val, 25) for key, val in lda_accuracy.items()}
q75_lda_accuracy = {key: np.percentile(val, 75) for key, val in lda_accuracy.items()}

q25_rf_accuracy = {key: np.percentile(val, 25) for key, val in rf_accuracy.items()}
q75_rf_accuracy = {key: np.percentile(val, 75) for key, val in rf_accuracy.items()}

q25_svm_accuracy = {key: np.percentile(val, 25) for key, val in svm_accuracy.items()}
q75_svm_accuracy = {key: np.percentile(val, 75) for key, val in svm_accuracy.items()}

mean_lda_accuracy = {key: np.mean(val) for key, val in lda_accuracy.items()}
mean_rf_accuracy = {key: np.mean(val) for key, val in rf_accuracy.items()}
mean_svm_accuracy = {key: np.mean(val) for key, val in svm_accuracy.items()}

# Plot accuracy comparison with 25th and 75th percentiles
# Create subplots for each classifier
#fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(8, 15))
sns.set_style('dark')
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(25,7))
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
# LDA accuracy plot
ax1.plot(snrs, mean_lda_accuracy.values(), '-o', label='Mean (n = 6)')
ax1.plot(snrs, q25_lda_accuracy.values(), '--o', label='25th percentile', color='g')
ax1.plot(snrs, q75_lda_accuracy.values(), '--o', label='75th percentile', color='r')
ax1.plot(snrs, [lda_baseline_acc]*len(snrs), linestyle='dashed', label='Accuracy (without noise)')
ax1.set_xlabel('Signal-to-Noise Ratio', fontsize=15)
ax1.set_ylabel('Accuracy', fontsize=15)
ax1.set_title('Linear Discriminant Analysis', fontsize=20)
ax1.set_ylim([min_accuracy, max_accuracy]);
ax1.legend()

# RF accuracy plot
ax2.plot(snrs, mean_rf_accuracy.values(), '-o', label='Mean (n = 6)')
ax2.plot(snrs, q25_rf_accuracy.values(), '--o', label='25th percentile', color='g')
ax2.plot(snrs, q75_rf_accuracy.values(), '--o', label='75th percentile', color='r')
ax2.plot(snrs, [rf_baseline_acc]*len(snrs), linestyle='dashed', label='Accuracy (without noise)')
ax2.set_xlabel('Signal-to-Noise Ratio', fontsize=15)
ax2.set_ylabel('Accuracy', fontsize=15)
ax2.set_title('Random Forest', fontsize=20)
ax2.set_ylim([min_accuracy, max_accuracy]);
ax2.legend()

# SVM accuracy plot
ax3.plot(snrs, mean_svm_accuracy.values(), '-o', label='Mean (n = 6)')
ax3.plot(snrs, q25_svm_accuracy.values(), '--o', label='25th percentile', color='g')
ax3.plot(snrs, q75_svm_accuracy.values(), '--o', label='75th percentile', color='r')
ax3.plot(snrs, [svm_baseline_acc]*len(snrs), linestyle='dashed', label='Accuracy (without noise)')
ax3.set_xlabel('Signal-to-Noise Ratio', fontsize=15)
ax3.set_ylabel('Accuracy', fontsize=15)
ax3.set_title('Support Vector Machine', fontsize=20)
ax3.set_ylim([min_accuracy, max_accuracy]);
ax3.legend()

plt.tight_layout()
plt.show()

# Print summary of results
print("Mean accuracy for LDA:", mean_lda_accuracy)
print("Mean accuracy for RF:", mean_rf_accuracy)
print("Mean accuracy for SVM:", mean_svm_accuracy)

In [ ]:
import scipy.stats as ss
for ft in range(0, len(ft_imp_matrix)):
    ft_imp_matrix[ft] = ft_imp_matrix[ft].to_frame().rename(columns={0:'importance'})
    ft_imp_matrix[ft]['rank'] = ft_imp_matrix[ft]['importance'].rank(ascending=False)
    ft_imp_matrix[ft].drop(columns={'importance'}, inplace=True)
    ft_imp_matrix[ft] = ft_imp_matrix[ft].iloc[:, 0]

In [ ]:
ft_imp_df = pd.DataFrame(ft_imp_matrix, columns=features)

In [ ]:
ft_imp_df.dropna(axis=1, how='all').head(10)

In [ ]:
ft_imp_df.columns.values

## Plot SNR vs. Accuracy - OLD

In [ ]:



sns.set_style('dark')
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14,7))
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

min_accuracy = min(min(lda_accuracy), min(rf_accuracy), min(svm_accuracy)) - 0.01
max_accuracy = max(lda_baseline_acc, rf_baseline_acc, svm_baseline_acc) + 0.01

axs[0].plot(snrs, lda_accuracy, label='Accuracy (with noise)', marker='o');
axs[0].plot(snrs, [lda_baseline_acc]*len(snrs), linestyle='dashed', label='Accuracy (without noise)')
axs[0].set_title('Linear Discriminant Analysis', fontsize=20);
axs[0].set_xlabel('Signal to Noise Ratio', fontsize=15);
axs[0].set_ylabel('Classification Accuracy', fontsize=15);
axs[0].legend();
axs[0].set_ylim([min_accuracy, max_accuracy]);

axs[1].plot(snrs, rf_accuracy, label='Accuracy (with noise)', marker='o');
axs[1].plot(snrs, [rf_baseline_acc]*len(snrs), linestyle='dashed', label='Accuracy (without noise)')
axs[1].set_title('Random Forest', fontsize=20);
axs[1].set_xlabel('Signal to Noise Ratio', fontsize=15);
axs[1].legend();
axs[1].set_ylim([min_accuracy, max_accuracy]);

axs[2].plot(snrs, svm_accuracy, label='Accuracy (with noise)', marker='o');
axs[2].plot(snrs, [svm_baseline_acc]*len(snrs), linestyle='dashed', label='Accuracy (without noise)')
axs[2].set_title('Support Vector Machine', fontsize=20);
axs[2].set_xlabel('Signal to Noise Ratio', fontsize=15);
axs[2].legend();
axs[2].set_ylim([min_accuracy, max_accuracy]);

## Leave Feature Out Testing

In [ ]:
features_to_leave_out = ['EDA', 'Resp', 'ECG']
lda_accuracies = []
rf_accuracies = []
svm_accuracies = []
ft_imp_matrices = []
ft_imp_schmidts = []
for ftre in features_to_leave_out:
    snrs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6]
    lda_accuracy = []
    rf_accuracy = []
    svm_accuracy = []
    ft_imp_matrix = []
    ft_imp_schmidt = []
    # For each signal to noise ratio
    for i in range(len(snrs)):
        # Get data
        df = pd.read_csv('data/noise_snr_'+str(snrs[i])+'.csv', index_col=0)
        
        # Get columns to remove
        cols = ['Resp_C_rate']
        for cl in df.columns:
            if ftre in cl: cols.append(cl)
        
        # Since Resp_C_rate is null for the first four, simply get rid of it
        df = df.loc[:, ~df.columns.isin(cols)]

        # Get features, label
        features = df.drop('label', axis=1).columns
        X = df.drop('label', axis=1).values
        y = df['label'].values

        # Get train test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

        # Scale the data
        sc = StandardScaler()  
        X_train = sc.fit_transform(X_train)  
        X_test = sc.transform(X_test)  

        # Test LDA
        lda = LinearDiscriminantAnalysis()
        lda.fit(X_train, y_train)
        y_pred = lda.predict(X_test)
        lda_accuracy.append(accuracy_score(y_test, y_pred))

        # Feature importance the same way Schmidt did
        dt = DecisionTreeClassifier()
        dt.fit(X, y)

        feat_importance = dt.feature_importances_
        plt.figure(figsize=(40,20));
        tree.plot_tree(dt,fontsize=10, feature_names=features);
        plt.show();

        # Test RF
        classifier = RandomForestClassifier(max_depth=4, random_state=0)
        classifier.fit(X_train, y_train)  
        y_pred = classifier.predict(X_test)  
        rf_accuracy.append(accuracy_score(y_test, y_pred))
        importances = classifier.feature_importances_
        forest_importances = pd.Series(importances, index=features).sort_values(ascending=False)[:10]
        ft_imp_matrix.append(forest_importances)


        # Test SVM
        clf = SVC(kernel='linear', C=1, random_state=0)
        clf.fit(X_train, y_train)
        y_out = clf.predict(X_test)    
        svm_accuracy.append(accuracy_score(y_test, y_pred))
    
    # Append to outer lists
    lda_accuracies.append(lda_accuracy)
    rf_accuracies.append(rf_accuracy)
    svm_accuracies.append(svm_accuracy)
    ft_imp_matrices.append(ft_imp_matrix)
    ft_imp_schmidts.append(ft_imp_schmidt)

### Plot SNR vs. Accuracy for Leave Feature Out

In [ ]:
for idx in range(len(features_to_leave_out)):
    sns.set_style('dark')
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14,7))
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    
    print('Models trained without: ' + features_to_leave_out[idx])
    
    min_accuracy = min(min(lda_accuracies[idx]), min(rf_accuracies[idx]), min(svm_accuracies[idx])) - 0.01
    max_accuracy = max(lda_baseline_acc, rf_baseline_acc, svm_baseline_acc) + 0.01

    axs[0].plot(snrs, lda_accuracies[idx], label='Accuracy (with noise)', marker='o');
    axs[0].plot(snrs, [lda_baseline_acc]*len(snrs), linestyle='dashed', label='Accuracy (without noise)')
    axs[0].set_title('Linear Discriminant Analysis', fontsize=20);
    axs[0].set_xlabel('Signal to Noise Ratio', fontsize=15);
    axs[0].set_ylabel('Classification Accuracy', fontsize=15);
    axs[0].legend();
    axs[0].set_ylim([min_accuracy, max_accuracy]);

    axs[1].plot(snrs, rf_accuracies[idx], label='Accuracy (with noise)', marker='o');
    axs[1].plot(snrs, [rf_baseline_acc]*len(snrs), linestyle='dashed', label='Accuracy (without noise)')
    axs[1].set_title('Random Forest', fontsize=20);
    axs[1].set_xlabel('Signal to Noise Ratio', fontsize=15);
    axs[1].legend();
    axs[1].set_ylim([min_accuracy, max_accuracy]);

    axs[2].plot(snrs, svm_accuracies[idx], label='Accuracy (with noise)', marker='o');
    axs[2].plot(snrs, [svm_baseline_acc]*len(snrs), linestyle='dashed', label='Accuracy (without noise)')
    axs[2].set_title('Support Vector Machine', fontsize=20);
    axs[2].set_xlabel('Signal to Noise Ratio', fontsize=15);
    axs[2].legend();
    axs[2].set_ylim([min_accuracy, max_accuracy]);
    plt.show();